In [ ]:
"""
Author: Matt Martin
Date: 2/8/24
Desc: Demo using spark to transform data
"""

## create the spark connection/instance
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test") \
    .config("spark.sql.warehouse.dir", "./test_dw") \
    .getOrCreate()

In [ ]:
## generate some dummy data and write to parquet
#%pip install Faker

from faker import Faker
fake = Faker()
data = []
for i in range(0,1001):
    data.append({'first_name':fake.first_name(), 'last_name':fake.last_name(),'street_adrs':fake.street_address()
                 ,'zip_cd':fake.zipcode(), 'state_abrv':fake.state_abbr()})

df = spark.createDataFrame(data)

df.write.parquet('./dummy_data.parquet')

In [11]:
#task: Combine first and last name to one field and combine the address to one field separated by a dash

df1 = spark.read.parquet('./dummy_data.parquet')
df1.createOrReplaceTempView('people')


In [ ]:
## transform and write out
sql = """
    select concat(first_name, ' ',last_name) as full_name
        ,concat(street_adrs,'-',zip_cd,'-',state_abrv) as full_adrs
    from people
"""
dft = spark.sql(sql)
dft.show(5)

In [17]:
dft.write.parquet('./people_tsfm.parquet')

In [20]:
spark.stop()